In [1]:
!pip install xlsxwriter

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [11]:
ncd_drv7 = pd.read_excel('/Users/user/Documents/업무문서/ncd_drv7.xlsx')
boa7 = pd.read_excel('/Users/user/Documents/업무문서/boa7.xlsx')
boa_drv7 = boa7[boa7['상품명'].str[0:7] == '웍스 드라이브']

In [13]:
boa_drv7

,회원번호,회원명,구분1,구분2,구분3,상품명,paid user 여부,매출금액(당월),매출금액(전월),매출금액(전월대비증감),...,이용금액(당월),이용금액(전월),이용금액(전월대비증감),이용금액(전월대비%),파트너명,법인개인구분,플랫폼,리전명,리전번호,채널
7,3355516,스냅스타,Business Application,Business Applications,WORKBOX,웍스 드라이브 Standard Plus,paid,1.762200e+06,0.000000,1.762200e+06,...,2055960.0,0,2055960.0,-,(주)안랩클라우드메이트,법인,민간,한국,1,파트너 매출
182,3077352,롯데이노베이트(주),Business Application,Business Applications,WORKBOX,웍스 드라이브 Lite,paid,1.308000e+05,0.000000,1.308000e+05,...,174450.0,0,174450.0,-,-,법인,민간,한국,1,온라인 매출
280,3230410,주식화시 우솔교육,Business Application,Business Applications,WORKBOX,웍스 드라이브 Lite,paid,9.396351e+04,0.000000,9.396351e+04,...,139220.0,0,139220.0,-,-,법인,민간,한국,1,온라인 매출
456,2654973,주식회사두성,Business Application,Business Applications,WORKBOX,웍스 드라이브 Standard Plus,paid,6.235538e+04,0.000000,6.235538e+04,...,80830.0,0,80830.0,-,-,법인,민간,한국,1,온라인 매출
673,3356732,인하대학교 산학협력단,Business Application,Business Applications,WORKBOX,웍스 드라이브 Lite,paid,4.570000e+04,0.000000,4.570000e+04,...,61030.0,0,61030.0,-,-,법인,민간,한국,1,온라인 매출
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24683,3361895,뉴경동관광(주),Business Application,Business Applications,NAVER WORKS,웍스 드라이브 문서 공동 편집,paid,6.893617e+02,0.000000,6.893617e+02,...,720.0,0,720.0,-,-,법인,민간,한국,1,온라인 매출
30031,2659015,우크반,Business Application,Business Applications,WORKBOX,웍스 드라이브 Standard Plus,paid,6.000000e+03,5400.000000,6.000000e+02,...,7000.0,6300,700.0,0.111111,-,법인,민간,한국,1,온라인 매출
32843,2904285,김**,Business Application,Business Applications,WORKBOX,웍스 드라이브 Lite,paid,1.433140e+03,1583.216794,-1.500773e+02,...,1800.0,1730,70.0,0.040462,-,개인,민간,한국,1,온라인 매출
32878,2904285,김**,Business Application,Business Applications,WORKBOX,웍스 드라이브 Standard,-,0.000000e+00,173.879301,-1.738793e+02,...,0.0,190,-190.0,-1,-,개인,민간,한국,1,-


In [83]:
drv_merged = pd.merge(ncd_drv7, boa_drv7, how='inner', left_on=['회원번호', '미터링유형코드명'], right_on=['회원번호', '상품명'])
drv_merged['할인금액'] = drv_merged['이용금액(당월)'] - drv_merged['매출금액(당월)']
drv_merged['할인율'] = drv_merged['할인금액']/drv_merged['이용금액(당월)']
drv_merged['dayCount'] = drv_merged.groupby(['회원번호', '상품명'])['회원번호'].transform('size')
drv_merged['일매출(당일)'] = (drv_merged['요금']/31)*drv_merged['사용량']*(1-drv_merged['할인율'])
drv_merged

,회원번호,회원그룹코드,회원그룹명,파트너사회원번호,청구유형코드,청구유형코드명,청구유형상세코드,청구유형상세코드명,인스턴스유형코드,인스턴스유형코드명,...,파트너명,법인개인구분,플랫폼,리전명_y,리전번호_y,채널,할인금액,할인율,dayCount,일매출(당일)
0,2611469,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,-,개인,민간,한국,1,온라인 매출,1000.0,0.142857,31,193.548387
1,2611469,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,-,개인,민간,한국,1,온라인 매출,1000.0,0.142857,31,193.548387
2,2611469,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,-,개인,민간,한국,1,온라인 매출,1000.0,0.142857,31,193.548387
3,2611469,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,-,개인,민간,한국,1,온라인 매출,1000.0,0.142857,31,193.548387
4,2611469,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_SPLUS_USER_CNT,웍스 드라이브 Standard Plus,WORK2,NAVER WORKS,...,-,개인,민간,한국,1,온라인 매출,1000.0,0.142857,31,193.548387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1252,3380458,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_LITE_USER_CNT,웍스 드라이브 Lite,WORK2,NAVER WORKS,...,-,법인,민간,한국,1,온라인 매출,40.0,0.025974,5,251.361542
1253,3380458,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_LITE_USER_CNT,웍스 드라이브 Lite,WORK2,NAVER WORKS,...,-,법인,민간,한국,1,온라인 매출,40.0,0.025974,5,251.361542
1254,3380458,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_LITE_USER_CNT,웍스 드라이브 Lite,WORK2,NAVER WORKS,...,-,법인,민간,한국,1,온라인 매출,40.0,0.025974,5,251.361542
1255,3380458,GEN,Ncloud온라인회원,NaN,WORK2,NAVER WORKS,DRIVE_LITE_USER_CNT,웍스 드라이브 Lite,WORK2,NAVER WORKS,...,-,법인,민간,한국,1,온라인 매출,40.0,0.025974,5,251.361542


In [85]:
drv_merged['미터링일시'] = pd.to_datetime(drv_merged['미터링일시'])

drv_merged = drv_merged.sort_values(by=['회원번호', '상품명', '미터링일시'])

drv_merged['일매출(전일)'] = drv_merged.groupby(['회원번호', '상품명'])['일매출(당일)'].shift(1)

drv_merged['일매출 증가율'] = ((drv_merged['일매출(당일)'] - drv_merged['일매출(전일)']) / drv_merged['일매출(전일)'])

drv_merged['월매출(당월)'] = drv_merged['일매출(당일)'] * drv_merged['dayCount']
drv_merged = drv_merged.drop(['dayCount'], axis = 1)

In [89]:
drv_merged.to_excel('/Users/user/Documents/업무문서/union7.xlsx')